# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import openweathermapy.core as owm

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

607

In [3]:
# Create settings dictionary with information we're interested in

settings = {"units": "imperial", "appid": api_keys}

#dataframe to store API calls

WeatherPy_df =pd.DataFrame(columns = ["City","Cloudiness", "Country","Date", "Max Temp", "Wind Speed", 
                                        "Lat","Lng", "Humidity"
                                     ]
                          )

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
count = 1 # counter to count the number of calls to API
err = 0 # error count to record how many cities the API responds back with no data.
sets = 0 # counter to count the number of call sets
index = 0 # index for the data frame WeatherPy data frame

#Get current time
t0 =time.time()
print(f'Beginning Data Retrieval \n ........................')

for city in cities:
    try:
        city1 = city.replace(" ", "%20") # remove space in city name so the URL retrieved is not broken.
        print(f'Processing Record {count} of Set {sets} | {city}' )
        print(f'{owm.BASE_URL}weather?q={city1}&units=imperial&APPID={api_keys}')
        city_stats = owm.get_current(city, **settings)
        WeatherPy_df.set_value(index, "City", city_stats["name"])
        WeatherPy_df.set_value(index, "Cloudiness", city_stats("clouds.all"))
        WeatherPy_df.set_value(index, "Country", city_stats("sys.country"))
        WeatherPy_df.set_value(index, "Date", city_stats("dt"))
        WeatherPy_df.set_value(index, "Max Temp", city_stats("main.temp_max"))
        WeatherPy_df.set_value(index, "Wind Speed", city_stats("wind.speed"))
        WeatherPy_df.set_value(index, "Lat", city_stats("coord.lat"))
        WeatherPy_df.set_value(index, "Lng", city_stats("coord.lon"))
        WeatherPy_df.set_value(index, "Humidity", city_stats("main.humidity"))
        index = index +1 
    except Exception as e:
        print(e)
        err= err+1
    count = count+1
    
    if count == 51: # limit 50 API calls per minute
        t1=time.time()
        count = 1 
        sets = sets+1
        time.sleep(60-(t1-t0))
        t0=time.time()

print("end of stats loop")
print(f'Could not get data for {err} cities')

Beginning Data Retrieval 
 ........................
Processing Record 1 of Set 0 | ushuaia
http://api.openweathermap.org/data/2.5/weather?q=ushuaia&units=imperial&APPID=<module 'api_keys' from 'C:\\Users\\ebjoy\\Desktop\\Homework06---Python-APIs\\api_keys.py'>
HTTP Error 401: Unauthorized
Processing Record 2 of Set 0 | dikson
http://api.openweathermap.org/data/2.5/weather?q=dikson&units=imperial&APPID=<module 'api_keys' from 'C:\\Users\\ebjoy\\Desktop\\Homework06---Python-APIs\\api_keys.py'>
HTTP Error 401: Unauthorized
Processing Record 3 of Set 0 | faanui
http://api.openweathermap.org/data/2.5/weather?q=faanui&units=imperial&APPID=<module 'api_keys' from 'C:\\Users\\ebjoy\\Desktop\\Homework06---Python-APIs\\api_keys.py'>
HTTP Error 401: Unauthorized
Processing Record 4 of Set 0 | bontang
http://api.openweathermap.org/data/2.5/weather?q=bontang&units=imperial&APPID=<module 'api_keys' from 'C:\\Users\\ebjoy\\Desktop\\Homework06---Python-APIs\\api_keys.py'>
HTTP Error 401: Unauthorized


Processing Record 36 of Set 0 | harper
http://api.openweathermap.org/data/2.5/weather?q=harper&units=imperial&APPID=<module 'api_keys' from 'C:\\Users\\ebjoy\\Desktop\\Homework06---Python-APIs\\api_keys.py'>
HTTP Error 401: Unauthorized
Processing Record 37 of Set 0 | port alfred
http://api.openweathermap.org/data/2.5/weather?q=port%20alfred&units=imperial&APPID=<module 'api_keys' from 'C:\\Users\\ebjoy\\Desktop\\Homework06---Python-APIs\\api_keys.py'>
HTTP Error 401: Unauthorized
Processing Record 38 of Set 0 | bluff
http://api.openweathermap.org/data/2.5/weather?q=bluff&units=imperial&APPID=<module 'api_keys' from 'C:\\Users\\ebjoy\\Desktop\\Homework06---Python-APIs\\api_keys.py'>
HTTP Error 401: Unauthorized
Processing Record 39 of Set 0 | turochak
http://api.openweathermap.org/data/2.5/weather?q=turochak&units=imperial&APPID=<module 'api_keys' from 'C:\\Users\\ebjoy\\Desktop\\Homework06---Python-APIs\\api_keys.py'>
HTTP Error 401: Unauthorized
Processing Record 40 of Set 0 | utiroa

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot